In [2]:
from dotenv import load_dotenv
import os
from pydantic import BaseModel
from langchain_openai import OpenAI  
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
#from langchain_community.graphs import Neo4jGraph
import traceback
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError
import requests
import fitz 

In [3]:
# --- Get OpenAI key

load_dotenv()

True

In [4]:

# --- Neo4j Tool
NEO4J_URI="bolt://3.95.30.19"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="departures-octobers-justice"
NEO4J_DATABASE="neo4j"

URI = "bolt://44.192.43.48:7687"
AUTH = ("neo4j", "deeds-combs-rays")
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
driver.verify_connectivity()
print(" Connected to Neo4j!")
with driver.session() as session:
    results = session.run("MATCH (n) RETURN n LIMIT 5").data()
print(results)

 Connected to Neo4j!
[{'n': {'gender': 'F', 'reportDate': neo4j.time.Date(2021, 8, 27), 'ageUnit': 'YR', 'reporterOccupation': 'Physician', 'primaryid': 100654764, 'age': 56.0, 'eventDate': neo4j.time.Date(2014, 3, 25)}}, {'n': {'gender': 'F', 'reportDate': neo4j.time.Date(2021, 9, 13), 'ageUnit': 'YR', 'reporterOccupation': 'Physician', 'primaryid': 101678656, 'age': 53.0, 'eventDate': neo4j.time.Date(2014, 5, 3)}}, {'n': {'gender': 'M', 'reportDate': neo4j.time.Date(2021, 8, 25), 'ageUnit': 'YR', 'reporterOccupation': 'Hospital', 'primaryid': 101980392, 'age': 77.0, 'eventDate': neo4j.time.Date(2010, 12, 27)}}, {'n': {'gender': 'M', 'reportDate': neo4j.time.Date(2021, 7, 20), 'ageUnit': 'YR', 'reporterOccupation': 'Hospital', 'primaryid': 103011729, 'age': 60.0, 'eventDate': neo4j.time.Date(2013, 6, 14)}}, {'n': {'gender': 'M', 'reportDate': neo4j.time.Date(2021, 9, 25), 'ageUnit': 'YR', 'reporterOccupation': 'Pharmacist', 'primaryid': 103829296, 'age': 31.0, 'eventDate': neo4j.time.

In [5]:
# def get_schema(tx):
#     # Consulta para pegar os tipos de nós e relacionamentos
#     result = tx.run("""
#         CALL db.schema.visualization()
#     """)
#     return result.single()

# with driver.session() as session:
#     schema = session.read_transaction(get_schema)


In [6]:
# # https://python.langchain.com/docs/integrations/graphs/neo4j_cypher/

# graph = Neo4jGraph(
#     url=NEO4J_URI,
#     username=NEO4J_USERNAME,
#     password=NEO4J_PASSWORD
# )

# # Get Neo4j Database Schema
# schema = graph.schema
# print(graph.schema)

In [7]:
# # Get Neo4j Database Schema
# schema = graph.schema

# # Seu prompt original para geração de Cypher
# CYPHER_GENERATION_PROMPT = PromptTemplate(
#     input_variables=["schema", "question"],
#     template="""Task:Generate Cypher statement to query a graph database.
# Instructions:
# Use only the provided relationship types and properties in the schema.
# Do not use any other relationship types or properties that are not provided.
# After identifying the drug to which the question is related to, capitalize all letters of it.

# Schema:
# {schema}

# Write multiple Cypher queries , corresponding to all of the possible paths between nodes, if multiple are possible.
# Example - connecting Manufacturer with Drug:
# 1) (m:Manufacturer)-[:REGISTERED]->(c:Case)-[:IS_PRIMARY_SUSPECT]->(d:Drug)
# 2) (m:Manufacturer)-[:REGISTERED]->(c:Case)-[:IS_CONCOMITANT]->(d:Drug)
# 3) (m:Manufacturer)-[:REGISTERED]->(:Case)-[:IS_SECONDARY_SUSPECT]->(:Drug)

# Note: Do not include any explanations or apologies in your responses.
# Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
# Do not include any text except the generated Cypher statement.

# The question is:
# {question}"""
# )

# # Prompt para geração da resposta com contexto
# QA_PROMPT = PromptTemplate(
#     input_variables=["question", "context"],
#     template="""
# Use apenas as informações em 'context' para responder.
# Se 'context' estiver vazio, responda: "Não sei".

# Context:
# {context}

# Pergunta: {question}
# Resposta:""",
# )

# # Inicializa o chain com os ajustes:
# chain = GraphCypherQAChain.from_llm(
#     cypher_llm=ChatOpenAI(temperature=0, model="gpt-4.1-nano-2025-04-14"),
#     qa_llm=ChatOpenAI(temperature=0, model="gpt-4.1-nano-2025-04-14"),
#     graph=graph,
#     verbose=True,
#     cypher_prompt=CYPHER_GENERATION_PROMPT,
#     qa_prompt=QA_PROMPT,
#     allow_dangerous_requests=True,
#     return_direct=True, # Avoids model from replying idk even if it gets right context
#     return_intermediate_steps=True,
#     top_k=10, # Limits results,
#     validate_cypher=True  # Prevents wrongful cypher syntax/queries
# )

# # Executa a consulta
# question = "Which manufacturers are connected to drugs which contain furoSemide in its name?"
# result = chain.invoke({"query": question})

# # Outputs
# print("👉 Result completo:")
# print(result)

# print("\n🔹 Intermediate steps:")
# for step in result["intermediate_steps"]:
#     print(step)

# print("\n✅ Result direto (lista JSON):")
# print(result["result"])


In [8]:
# --- Neo4j Tool
NEO4J_URI="bolt://3.95.30.19"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="departures-octobers-justice"
NEO4J_DATABASE="neo4j"

# URI = "bolt://44.192.43.48:7687"
# AUTH = ("neo4j", "deeds-combs-rays")
# driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
# driver.verify_connectivity()
# print(" Connected to Neo4j!")
# with driver.session() as session:
#     results = session.run("MATCH (n) RETURN n LIMIT 5").data()
# print(results)

# Connect to Neo4j DB
graph = Neo4jGraph(
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD
    )

# Get Neo4j Database Schema
schema = graph.schema

# Seu prompt original para geração de Cypher
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"],
    template="""Task:Generate Cypher statement to query a graph database.
    Instructions:
    Use only the provided relationship types and properties in the schema.
    Do not use any other relationship types or properties that are not provided.
    After identifying the drug to which the question is related to, capitalize all letters of it.

    Schema:
    {schema}

    Write multiple Cypher queries , corresponding to all of the possible paths between nodes, if multiple are possible.
    Example - connecting Manufacturer with Drug:
    1) (m:Manufacturer)-[:REGISTERED]->(c:Case)-[:IS_PRIMARY_SUSPECT]->(d:Drug)
    2) (m:Manufacturer)-[:REGISTERED]->(c:Case)-[:IS_CONCOMITANT]->(d:Drug)
    3) (m:Manufacturer)-[:REGISTERED]->(:Case)-[:IS_SECONDARY_SUSPECT]->(:Drug)

    Note: Do not include any explanations or apologies in your responses.
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    Do not include any text except the generated Cypher statement.

    The question is:
    {question}"""
    )

# Prompt para geração da resposta com contexto
QA_PROMPT = PromptTemplate(
    input_variables=["question", "context"],
    template="""
    Use apenas as informações em 'context' para responder.
    Se 'context' estiver vazio, responda: "Não sei".

    Context:
    {context}

    Pergunta: {question}
    Resposta:""",
    )

# Inicializa o chain com os ajustes:
chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-4.1-nano-2025-04-14"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-4.1-nano-2025-04-14"),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    qa_prompt=QA_PROMPT,
    allow_dangerous_requests=True,
    return_direct=True, # Avoids model from replying idk even if it gets right context
    return_intermediate_steps=True,
    top_k=10, # Limits results,
    validate_cypher=True,  # Prevents wrongful cypher syntax/queries
    timeout=10 # Raises connection timeouts
    )


class Neo4jQueryInput(BaseModel):
    question: str

from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError

def query_neo4j(question: str) -> str:
    return chain.invoke({"query": question})


neo4j_tool = StructuredTool.from_function(
    func=query_neo4j,
    name="Neo4jTool",
    description="""Answers questions about the Neo4j Healthcare Analytics template,
                   which contains information about Drugs, Manufacturers, Therapies, Outcomes, and Reactions.
                   After identifying the drug to which the question is related to, capitalize all letters of it. Example: furosemide -> FUROSEMIDE this is Very Important!!! 
                   Example question: Which manufacturers are connected to drugs that contain Furosemide in its name?
                   
                   Output Restrictions:
                   The queries are limited to 10 results, for cost management purposes.""",
    args_schema=Neo4jQueryInput,
)

In [9]:
print(graph.schema)

Node properties:
Drug {primarySubstabce: STRING, name: STRING}
Case {reporterOccupation: STRING, gender: STRING, eventDate: DATE, reportDate: DATE, primaryid: INTEGER, age: FLOAT, ageUnit: STRING}
Reaction {description: STRING}
ReportSource {code: STRING, name: STRING}
Outcome {outcome: STRING, code: STRING}
Therapy {primaryid: INTEGER}
Manufacturer {manufacturerName: STRING}
AgeGroup {ageGroup: STRING}
Relationship properties:
IS_PRIMARY_SUSPECT {indication: STRING, doseAmount: STRING, doseUnit: STRING, drugSequence: STRING, route: STRING}
IS_SECONDARY_SUSPECT {indication: STRING, doseAmount: STRING, doseUnit: STRING, drugSequence: STRING, route: STRING}
IS_CONCOMITANT {indication: STRING, doseAmount: STRING, doseUnit: STRING, drugSequence: STRING, route: STRING}
IS_INTERACTING {route: STRING, indication: STRING, doseAmount: STRING, doseUnit: STRING, drugSequence: STRING}
PRESCRIBED {drugSequence: STRING, endYear: STRING, startYear: STRING}
The relationships:
(:Case)-[:RECEIVED]->(:Th

In [10]:
# --- Adverse Events Tool

def get_adverse_events(drug_name: str, limit: int = 100) -> str:
    drug_name = drug_name.upper()
    url = f"https://api.fda.gov/drug/event.json?search=patient.drug.medicinalproduct:{drug_name}&limit={limit}"
    response = requests.get(url)
    data = response.json()

    adverse_effects_counter = {}
    for patient in data.get("results", []):
        for reaction in patient["patient"]["reaction"]:
            name = reaction["reactionmeddrapt"]
            adverse_effects_counter[name] = adverse_effects_counter.get(name, 0) + 1

    top_20 = sorted(adverse_effects_counter.items(), key=lambda x: x[1], reverse=True)[:10]
    formatted = "\n".join(f"{effect}: {count}" for effect, count in top_20)

    return f"Top 10 adverse effects for {drug_name}:\n{formatted}"

class AdverseEventsInput(BaseModel):
    drug_name: str

adverse_events_tool = StructuredTool.from_function(
    func=get_adverse_events,
    name="AdverseEventsRetriever",
    description="Gets the top 10 FDA adverse events for a given drug name.",
    args_schema=AdverseEventsInput,
)

In [11]:
def extract_text_from_pages(pdf_path:str, pages:list):
    """
    Extracts plain text from specific pages in a PDF.

    :param pdf_path: str – path to your PDF file
    :param pages: list of ints – page numbers you want (1-based)
    :return: dict mapping page number → extracted text
    """
    doc = fitz.open(pdf_path)
    result = {}

    for human_page in pages:
        idx = human_page - 1  # convert to zero-based index
        if idx < 0 or idx >= doc.page_count:
            print(f"⚠️ Page {human_page} out of range. Skipping.")
            continue

        page = doc.load_page(idx)  # get the page
        text = page.get_text("text")  # extract plain text
        result[human_page] = text

    doc.close()
    return result

def financial_report():
    # Extract text from main financial analysis pages in the 24/25 annual report (contains P&L)
    financial_context = extract_text_from_pages("../docs/grunenthal-annual-report24-25.pdf", [30,31,32])
    return financial_context


financial_report_tool = StructuredTool.from_function(
    func=financial_report,
    name="FinancialReportRetriever",
    description="Gets information about Grunenthal's financial report 24/25",
)

In [16]:
# ----- LLM and Agent

role_description = """
You are an experienced pharmaceutical data agent with access to three specialized tools:
1. **Neo4jTool** – for querying drug-related data (manufacturers, therapies, outcomes, reactions) from a Neo4j healthcare analytics graph.
2. **AdverseEventsRetriever** – for retrieving the top 10 FDA adverse events for a given drug via FDA API.
3. **FinancialReportRetriever** – for retrieving Grunenthal's financial report data 24/25 and overall company performance.

When a user asks about drug relationships (e.g. manufacturers, therapies, outcomes), use **Neo4jTool**.
When a user asks about adverse effects, use **AdverseEventsRetriever** and pass the drug name in UPPERCASE. Example: Tylenol -> TYLENOL
When a user asks about Grunenthal, use **FinancialReportRetriever**.  VERY IMPORTANT!!

Be very direct and precise in answering the question, reply only to what the user asked, don't add additional information. THIS IS VERY IMPORTANT!!! 

If the question is beyond your tools’ scope, reply exactly: 
"I can't help you with that. I'm equipped with relevant pharmaceutical data to provide you with evidence‑based insights in this field."
"""


agent = initialize_agent(
    tools=[neo4j_tool, adverse_events_tool, financial_report_tool],
    llm=OpenAI(temperature=0.02, model="gpt-4.1-nano-2025-04-14"),
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,  # importante,
    timeout=60,
    agent_kwargs={
        "prefix": role_description,
    },
)

MAX_RETRIES = 10
TIMEOUT_SECONDS = 60


def safe_run_agent(query: str, max_retries: int = MAX_RETRIES, timeout: int = TIMEOUT_SECONDS):
    """
    Executes the agent with a retry mechanism and timeout protection.

    Parameters:
        query (str): The input question or command to send to the agent.
        max_retries (int): The maximum number of attempts to retry the agent execution.
        timeout (int): Timeout duration in seconds for each attempt.

    Returns:
        dict: The result returned by the agent, including 'output' and optionally 'intermediate_steps'.

    Raises:
        RuntimeError: If all attempts fail, including due to timeout or unexpected exceptions.
    """
    for attempt in range(1, max_retries + 1):
        print(f"\nAttempt {attempt} of {max_retries}...")

        try:
            # Execute the agent in a separate thread to apply a timeout
            with ThreadPoolExecutor(max_workers=1) as executor:
                future = executor.submit(agent, {"input": query})
                result = future.result(timeout=timeout)

            # Check if the agent returned a valid response
            if not result.get("output"):
                print("Invalid or empty response. Retrying...")
                continue

            print("Execution successful.")
            return result

        except FutureTimeoutError:
            print(f"Execution timed out after {timeout} seconds. Retrying...")
        except Exception as e:
            print("Error occurred during agent execution:")
            traceback.print_exc()

    # All retries failed
    raise RuntimeError("All attempts to execute the agent have failed.")

# ----- Examples:

# -- Neo4j Tool
#query = "Give me example of three case that are connected to drugs that contain tylenol in its name? People between 30 and 40 years old." # Works fairly well.
#query = "What are typical dose amounts for Tramadol?" #- Rarely works. Relationship properties are hard for the LLM.
#query = "What are typical routes for Tramadol administration?" # Rarely works. Relationship properties are hard for the LLM.
#query = "Which manufacturers are connected to drugs which contain Tylenol in its name?" #- Works consistently.


# -- FDA Tool
#query = "“What are the top 10 most recent adverse events registered for a drug containing Tramadol in its name?"

# -- Financial Report Tool
# query = "Grunenthal's Revenue 2023 and 2024" # When year is specified, Model is able to find correct answer
# query = "Which year was the highest EBITDA for Grunenthal? 2023 or 2024" # When year is specified, Model is able to find correct answer
query = "Which year had the highest EBITDA for Grunenthal?" # When year is NOT specified, Model is NOT able to find correct answer
# query = "What is Fabian's opinion for Grunenthal's financial year?" # Model mixes direct quote with other document info - not ideal
# query = "What is Fabian's opinion for Grunenthal's year?" # Model refuses to answer - not clear why based on role prompt

# -- Guardrails test
#query = "Will Arsenal ever win the league again?" # Works

result = safe_run_agent(query)

# Saída formatada
print("\n🎯 Resposta final:", result["output"])
print("\n🧠 Pensamentos e ações:")
for action, obs in result["intermediate_steps"]:
    print(f"- Pensou: usar {action.tool} com input {action.tool_input}")
    print(f"  → Resultado: {obs}")



Attempt 1 of 10...


> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "FinancialReportRetriever",
  "action_input": {}
}
```

Observation: {30: "30\nGrünenthal Report 2024/2025\nSTRONG FINANCIAL PERFORMANCE \nResilience and stability\nGrünenthal's financial performance in 2024 \ndemonstrated resilience and stability, navigating \nchallenges and capitalising on opportunities to \nmaintain robust results. Despite the ongoing ge\xad\nneric erosion of Palexia™, one of our key brands, \nthree factors helped to offset this impact: the \ngrowth of Qutenza™, the strong development \nof our established brands, and the continued \nbolstering of our portfolio with strategic acqui\xad\nsitions, including Valinor Pharma and its product \nMovantik™.\nOur brands continue to drive \ngrowth\nThe continuous growth of Qutenza™ in the US \nand European markets, as well as the solid per\xad\nformance of the Grünenthal Meds joint venture \n(JV) and the remaining established brands port\x

In [13]:
# def get_adverse_events(drug_name: str, limit: int = 1000):
    
#     drug_name = drug_name.upper()
#     # Query adverse events API and get info for 1000 patients
#     url = (
#     f"https://api.fda.gov/drug/event.json?"
#     f"search=patient.drug.medicinalproduct:{drug_name}&limit={limit}"
#     )
#     response = requests.get(url)
#     adverse_events = response.json()

#     # Count adverse effects
#     adverse_effects_counter = {}

#     for idx, patient in enumerate(adverse_events["results"]):
#         for reaction in patient["patient"]["reaction"]:
#             reaction_name = reaction["reactionmeddrapt"]
#             if reaction_name in adverse_effects_counter.keys():
#                 adverse_effects_counter[reaction_name] += 1
#             else:
#                 adverse_effects_counter[reaction_name] = 1

#     # Get top 20 most common adverse effects
#     top_20_adverse_effects = dict(
#         sorted(adverse_effects_counter.items(), key=lambda item: item[1], reverse=True)[:20]
#     )

#     return top_20_adverse_effects

In [14]:
# # Count adverse effects
# adverse_effects_counter = {}
# # Confirm every patient received tramadol
# for idx, patient in enumerate(adverse_events["results"]):
#     print(f"Patient {idx} takes {len(patient["patient"]["drug"])} drugs")
#     for drug in patient["patient"]["drug"]:
#         if "TRAMADOL" in drug["medicinalproduct"].upper():
#             print(f"Patient {idx} uses tramadol")

#     for reaction in patient["patient"]["reaction"]:
#         reaction_name = reaction["reactionmeddrapt"]
#         if reaction_name in adverse_effects_counter.keys():
#             adverse_effects_counter[reaction_name] += 1
#         else:
#             adverse_effects_counter[reaction_name] = 1



In [15]:
# import fitz 
# def extract_text_from_pages(pdf_path:str, pages:list):
#     """
#     Extracts plain text from specific pages in a PDF.

#     :param pdf_path: str – path to your PDF file
#     :param pages: list of ints – page numbers you want (1-based)
#     :return: dict mapping page number → extracted text
#     """
#     doc = fitz.open(pdf_path)
#     result = {}

#     for human_page in pages:
#         idx = human_page - 1  # convert to zero-based index
#         if idx < 0 or idx >= doc.page_count:
#             print(f"⚠️ Page {human_page} out of range. Skipping.")
#             continue

#         page = doc.load_page(idx)  # get the page
#         text = page.get_text("text")  # extract plain text
#         result[human_page] = text

#     doc.close()
#     return result

# def financial_report():
#     # Extract text from main financial analysis pages in the 24/25 annual report (contains P&L)
#     financial_context = extract_text_from_pages("../docs/grunenthal-annual-report24-25.pdf", [30,31,32])
#     return financial_context


# financial_report_tool = StructuredTool.from_function(
#     func=financial_report,
#     name="FinancialReportRetriever",
#     description="Gets information about Grunenthal's financial report 24/25",
# )
